In [2]:
%pylab tk

import numpy as np
import mahotas as mh
import os
import cv2
import json
from time import time
from math import radians
from skimage import io, color, img_as_float, img_as_ubyte
from skimage.transform import rescale
from skimage.util import pad
from IPython.parallel import Client
from sklearn.decomposition import PCA, RandomizedPCA
from scipy.cluster.vq import kmeans2
from sklearn.cluster import DBSCAN
from sklearn.svm import SVC

import helpers.features as fh
import helpers.display as dh

BASE_PATH = os.getcwd()
print("Current base path: {0}".format(BASE_PATH))
DATA_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_cut_2/'
TO_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_outlined/'
TEST_FILES = json.loads(open(BASE_PATH + '/Daten/2D/testfiles.json').read())

NUMBER_OF_SECTIONS = 150
FEATURE_FN = mh.features.pftas
FEATURE_FN_2 = mh.features.haralick

def split(image):
    return numpy.array_split(image, NUMBER_OF_SECTIONS, axis=1)

def join(images):
    return numpy.concatenate(images, axis=1)

def extract_windows(image):
    split_y = numpy.array_split(image, NUMBER_OF_SECTIONS, axis=0) 
    
    return list(map(split, split_y))

def extract_features(windows, image):
    features = []
    for windows_x in windows:
        for window in windows_x:
            f1 = FEATURE_FN(window).flatten()
            #f2 = FEATURE_FN_2(window).flatten()
            #features.append(np.concatenate((f1, f2)))
            features.append(f1)
    return np.array(features)

def extract_features_full_img(windows, image):
    features = []
    for windows_x in windows:
        f_x = []
        for window in windows_x:
            f_x.append(np.full((window.shape[0], window.shape[1], 169), FEATURE_FN(window).flatten()))
        features.append(f_x)
        
    features = list(map(join, features))
        
    return np.concatenate(features).reshape((image.shape[0] * image.shape[1], 169))

def calculate_distance_for_single_image(image):
    windows = extract_windows(image)
    features = extract_features(windows, image)
    return fh.normalize_each(features)

def append_locality(features):
    x = np.tile(np.arange(NUMBER_OF_SECTIONS) + 1, NUMBER_OF_SECTIONS)
    y = np.repeat(np.arange(NUMBER_OF_SECTIONS) + 1, NUMBER_OF_SECTIONS)
    locality = np.divide(np.array([ x, y ]).transpose(), float(NUMBER_OF_SECTIONS))
    
    return np.concatenate((features, locality), axis=1)

def do_clustering(image, features):
    #db = DBSCAN(eps=0.3, min_samples=25).fit(features)
    #labels = db.labels_
    centroids, labels = kmeans2(features, 4)
    return labels.reshape((image.shape[0], image.shape[1]))
    
def reduce_feature_complexity(features):
    pca = PCA(n_components=6)
    #pca = RandomizedPCA(n_components=10)
    reduced = pca.fit_transform(features)
    return reduced

Populating the interactive namespace from numpy and matplotlib
Current base path: C:\Users\Stefan\Dropbox\Masterarbeit


In [9]:
file = TEST_FILES[2]
rgb_image = io.imread(DATA_PATH + file['filename'])
fake_img = np.zeros((NUMBER_OF_SECTIONS, NUMBER_OF_SECTIONS, 3))

features = calculate_distance_for_single_image(rgb_image)

In [ ]:
dh.features(plt, np.zeros((NUMBER_OF_SECTIONS, NUMBER_OF_SECTIONS, 3)), features)

In [10]:
reduced = reduce_feature_complexity(features)
reduced = append_locality(reduced)

In [4]:
dh.features(plt, np.zeros((NUMBER_OF_SECTIONS, NUMBER_OF_SECTIONS, 3)), reduced)

NameError: name 'reduced' is not defined

In [11]:
start_time = time()
labels = do_clustering(np.zeros((NUMBER_OF_SECTIONS, NUMBER_OF_SECTIONS, 3)), reduced)
print('do_clustering: {0}s'.format(time() - start_time))

do_clustering: 0.028960943222s
